In [2]:
# structure de données 

file = input("Quel fichier ?")

from importations import *
from classes import * 

from iteration_valeurs import * 
from smc import *
from q_learning import *
from visualisation import visu_random_walk
from accessibilite import *

lexer = gramLexer(FileStream(file))
stream = CommonTokenStream(lexer)
parser = gramParser(stream)
tree = parser.program()
data_listener = gramDataListener()
walker = ParseTreeWalker()
walker.walk(data_listener, tree)

#création de states, actions, transitions
states = data_listener.states
actions = data_listener.actions
transitions = data_listener.transitions
statesid = [state.id for state in states]
statesgain = [state.gain for state in states]

print(f"Liste des états : {statesid}")
print(f"Liste des gains : {statesgain}")
print()
print(actions)
print()
for transition in transitions:
    print(transition)
print()

#Vérification : chaque transition est effectuée avec une action définie
for transition in transitions:
    if str(transition.action) not in str(actions) and transition.action.id != '':
        raise ValueError(f"{transition.action.id} n\'est pas une action valide")
    
#Vérification : chaque transition part d'un état bien défini vers un état bien défini
for transition in transitions:
    if str(transition.departure.id) not in str(statesid) :
                raise ValueError(f"{transition.departure.id} n\'est pas un état valide")
            
    for target in transition.targets : 
        if str(target.id) not in str(statesid) : 
                raise ValueError(f"{target.id} n\'est pas un état valide")
            
#Vérif pas déterministe et non-déterministe
l1 = []
l2 = []
for transition in transitions:
    if  transition.action.id != '':
        l1.append(transition.departure.id)
    else :
        l2.append(transition.departure.id)
intersection = [x for x in l2 if x in l1]
if len (intersection) !=0 : 
        raise ValueError(f"{intersection[0]} est à la fois déterministe et non-déterministe.")
        
#Vérif : chaque état à une tansition
e = []
for transition in transitions:
    if transition.departure.id not in e : 
        e.append(transition.departure.id)
if len(e) != len(states) : 
    l = [x.id for x in states if x.id not in e]
    raise ValueError(f"{l[0]} n'a pas de transition.")


ANTLR runtime and generated code versions disagree: 4.11.1!=4.12.0
ANTLR runtime and generated code versions disagree: 4.11.1!=4.12.0
Liste des états : ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6']
Liste des gains : ['', '', '', '', '', '', '', '', '', '', '', '', '']

[Action(None)]

Transition(State(S0), Action(), [State(S1), State(S2)], [5, 5])
Transition(State(S1), Action(), [State(S4), State(S3)], [5, 5])
Transition(State(S2), Action(), [State(S5), State(S6)], [5, 5])
Transition(State(S3), Action(), [State(S1), State(D1)], [5, 5])
Transition(State(S4), Action(), [State(D2), State(D3)], [5, 5])
Transition(State(S5), Action(), [State(D4), State(D5)], [5, 5])
Transition(State(S6), Action(), [State(D6), State(S2)], [5, 5])
Transition(State(D1), Action(), [State(D1)], [1])
Transition(State(D2), Action(), [State(D2)], [1])
Transition(State(D3), Action(), [State(D3)], [1])
Transition(State(D4), Action(), [State(D4)], [1])
Transition(State(D5), Action(), [

# Visualisation

In [3]:
visu_random_walk(states,actions,transitions) 

current_state : State(S0)
next_transitions : [Transition(State(S0), Action(), [State(S1), State(S2)], [5, 5])]
next_action : Action()
next_state : State(S2)
Chemin 1: départ de State(S0) et arrivée à State(S2)

current_state : State(S2)
next_transitions : [Transition(State(S2), Action(), [State(S5), State(S6)], [5, 5])]
next_action : Action()
next_state : State(S5)
Chemin 2: départ de State(S2) et arrivée à State(S5)

current_state : State(S5)
next_transitions : [Transition(State(S5), Action(), [State(D4), State(D5)], [5, 5])]
next_action : Action()
next_state : State(D5)
Chemin 3: départ de State(S5) et arrivée à State(D5)



# smc quantitatif 

In [15]:
#S = input("Quel état")
#n = input ("nombre itérations")
#eps = input("epsion")
#delt = input ("delta")

eps = 0.01
delt = 0.1
n = 5
S = 'D1'
#de.mdp

smc_quantitatif(S,n,eps,delt,states,actions,transitions)



0.14901198079039712


# smc qualitatif

In [11]:
#de.mdp

#smc_qualitatif (alpha,beta,theta,n,S, states, actions, transitions) 
smc_qualitatif(0.1,0.1,0.12,5,'D1',states,actions,transitions)




-0.9542425094393249 0.9542425094393249
plus grand que 0.12


# q_learning

In [23]:
q_learning(states,actions,transitions,100,1/2)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
mon état initial est 0
0 [0]
0
[[2. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
------------
[[2.5 0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]]

2 [0, 2]
2
[[2. 1. 1.]
 [1. 1. 1.]
 [1. 1. 2.]]
------------
[[2.5 0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]]

1 [0, 2, 1]
1
[[2. 1. 1.]
 [1. 2. 1.]
 [1. 1. 2.]]
------------
[[2.5 0.  0. ]
 [0.  0.5 0. ]
 [0.  0.  0. ]]

1 [2, 1, 1]
1
[[2. 1. 1.]
 [2. 2. 1.]
 [1. 1. 2.]]
------------
[[2.5   0.    0.   ]
 [0.625 0.5   0.   ]
 [0.    0.    0.   ]]

1 [1, 1, 1]
1
[[2. 1. 1.]
 [2. 3. 1.]
 [1. 1. 2.]]
------------
[[2.5        0.         0.        ]
 [0.625      1.08333333 0.        ]
 [0.         0.         0.        ]]

0 [1, 1, 0]
0
[[3. 1. 1.]
 [2. 3. 1.]
 [1. 1. 2.]]
------------
[[3.33333333 0.         0.        ]
 [0.625      1.08333333 0.        ]
 [0.         0.         0.        ]]

2 [1, 0, 2]
2
[[3. 1. 1.]
 [2. 3. 1.]
 [2. 1. 2.]]
------------
[[3.33333333 0.         0.        ]
 [0.625      1.08333333 0.        ]
 [0

array([[5.38044227, 5.63112927, 5.37408524],
       [2.58570506, 2.60073201, 2.83993175],
       [1.25209031, 1.46557165, 1.32266826]])

# accessibilité MC

In [5]:
state = input ("quel état ?")
accessibilite(state,transitions, states)

S0 = ['D2', 'D3', 'D4', 'D5', 'D6', 'S4', 'S5', 'S2', 'S6'] ; S1 = ['D1'] ; S? = ['S0', 'S1', 'S3']


# accessibilite MDP

In [27]:
#state = input ("quel état ?")
#adversaire = input("quel adversaire")

visu_random_walk(states,actions,suppr(transitions,['a']*16,statesid)) 
accessibilite_mdp('S11',transitions,states,['a']*16)

current_state : State(S11)
next_transitions : [Transition(State(S11), Action(a), [State(S21), State(L), State(S22)], [3, 1, 1])]
next_action : Action(a)
next_state : State(S21)
Chemin 1: départ de State(S11) et arrivée à State(S21)

current_state : State(S21)
next_transitions : [Transition(State(S21), Action(a), [State(S31), State(L), State(S32)], [3, 1, 1])]
next_action : Action(a)
next_state : State(S31)
Chemin 2: départ de State(S21) et arrivée à State(S31)

current_state : State(S31)
next_transitions : [Transition(State(S31), Action(a), [State(S41), State(L), State(S42)], [3, 1, 1])]
next_action : Action(a)
next_state : State(S41)
Chemin 3: départ de State(S31) et arrivée à State(S41)

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
[Transition(State(S11), Action(a), [State(S21), State(L), State(S22)], [3, 1, 1]), Transition(State(S12), Action(a), [State(S22), State(S21), State(S23)], [3, 1, 1]), Transition(State(S13), Action(a), [State(S23), State(

# itération de valeurs

In [28]:
iteration_valeur(0.1,0.5,states,actions,transitions)

ValueError: could not convert string to float: ''